In [1]:
import math
import numpy as np
import pandas as pd
import os
import glob

from pykrx import stock
from pykrx import bond

import time
from time import sleep
from datetime import datetime
from datetime import timedelta

from pyarrow import csv
import pyarrow as pa
import pyarrow.parquet as pq

from ta.trend import MACD
from ta.momentum import StochasticOscillator

import psycopg2 as pg2
from sqlalchemy import create_engine

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage


import warnings
warnings.filterwarnings('ignore')

# 경로 변경
os.chdir('/home/shjj08choi4/finance_mlops')


# 서비스 계정 키 JSON 파일 경로
key_path = glob.glob("key_value/*.json")[0]

# Credentials 객체 생성
credentials = service_account.Credentials.from_service_account_file(key_path)

# 빅쿼리 정보
project_id = 'owenchoi-404302'
dataset_id = 'finance_mlops'


# BigQuery 클라이언트 객체 생성
client = bigquery.Client(credentials = credentials, 
                         project = credentials.project_id)


# GCP 클라이언트 객체 생성
storage_client = storage.Client(credentials = credentials,
                         project = credentials.project_id)
bucket_name = 'finance-mlops-proj'    # 서비스 계정 생성한 bucket 이름 입력


now = datetime.now()
# now = now + timedelta(days=-2)
today_date1 = now.strftime('%Y%m%d')
today_date2 = now.strftime('%Y-%m-%d')
today_date_time_csv = now.strftime("%Y%m%d_%H%M")

today_date1 = '20231226'
today_date2 = '2023-12-26'

In [247]:
now = datetime.now()
print(now)

2024-01-07 03:28:26.057347


In [130]:
sql = f"""
select 
  `date`,
  open, 
  high, 
  low, 
  close, 
  volume,
  price_change_percentage,
  `{project_id}.{dataset_id}.kor_stock_ohlcv`.ticker,
  corp_name, 
  market 
from `{project_id}.{dataset_id}.kor_stock_ohlcv`
left join  `{project_id}.{dataset_id}.kor_ticker_list`
on `{project_id}.{dataset_id}.kor_stock_ohlcv`.ticker = `{project_id}.{dataset_id}.kor_ticker_list`.ticker
where market = 'KOSPI'
order by date asc
"""

In [131]:
# 데이터 조회 쿼리 실행 결과
query_job = client.query(sql)

# 데이터프레임 변환
ohlcv_df_raw = query_job.to_dataframe()

In [132]:
ohlcv_df_raw = ohlcv_df_raw.fillna(0)
ticker_list = ohlcv_df_raw['ticker'].unique()

### 주가 지표

In [234]:
df_raw_total = pd.DataFrame()
df_raw_anal_total = pd.DataFrame()

for ticker_nm in ticker_list[:5]:
    df_raw = ohlcv_df_raw[ohlcv_df_raw['ticker'] == ticker_nm].reset_index(drop = True)

    ######################################################################
    # 보조지표
    ######################################################################

    # 이동평균선
    df_raw['MA5'] = df_raw['close'].rolling(window=5).mean()
    df_raw['MA20'] = df_raw['close'].rolling(window=20).mean()
    df_raw['MA60'] = df_raw['close'].rolling(window=60).mean()
    df_raw['MA120'] = df_raw['close'].rolling(window=120).mean()

    # 볼린저밴드
    std = df_raw['close'].rolling(20).std(ddof=0)

    df_raw['upper'] = df_raw['MA20'] + 2 * std
    df_raw['lower'] = df_raw['MA20'] - 2 * std

    # MACD
    # MACD
    macd = MACD(close=df_raw['close'],
                window_slow=26,
                window_fast=12,
                window_sign=9)


    df_raw['MACD_DIFF'] = macd.macd_diff()
    df_raw['MACD'] = macd.macd()
    df_raw['MACD_Signal'] = macd.macd_signal()

    # RSI
    df_raw['변화량'] = df_raw['close'] - df_raw['close'].shift(1)
    df_raw['변화량'] = df_raw['변화량'].astype('float64')
    df_raw['상승폭'] = np.where(df_raw['변화량']>=0, df_raw['변화량'], 0)
    df_raw['하락폭'] = np.where(df_raw['변화량'] <0, df_raw['변화량'].abs(), 0)

    # welles moving average
    df_raw['AU'] = df_raw['상승폭'].ewm(alpha=1/14, min_periods=14).mean()
    df_raw['AD'] = df_raw['하락폭'].ewm(alpha=1/14, min_periods=14).mean()
    df_raw['RSI'] = df_raw['AU'] / (df_raw['AU'] + df_raw['AD']) * 100

    df_raw['MA5-20'] = df_raw['MA5'] - df_raw['MA20']
    df_raw['MA20-60'] = df_raw['MA20'] - df_raw['MA60']
    df_raw['MA60-120'] = df_raw['MA60'] - df_raw['MA120']


    ######################################################################
    # 보조지표 분석
    ######################################################################
    df_raw_anal = df_raw[['date','ticker', 'corp_name','market', 'close']]

    # 골든크로스
    # 골든 크로스 5-20
    # 음수에서 양수로 바뀌는 모든 인덱스 찾기
    idx_5_20_gold_cross = [idx for idx in range(len(df_raw)) if df_raw["MA5-20"].iloc[idx] > 0 and df_raw["MA5-20"].iloc[idx - 1] <= 0]

    # 데드 크로스 5-20
    # 양수에서 음수로 바뀌는 모든 인덱스 찾기
    idx_5_20_dead_cross = [idx for idx in range(len(df_raw)) if df_raw["MA5-20"].iloc[idx] < 0 and df_raw["MA5-20"].iloc[idx - 1] >= 0]

    # 골든 크로스 20-60
    # 음수에서 양수로 바뀌는 모든 인덱스 찾기
    idx_20_60_gold_cross = [idx for idx in range(len(df_raw)) if df_raw["MA20-60"].iloc[idx] > 0 and df_raw["MA20-60"].iloc[idx - 1] <= 0]

    # 골든 크로스 20-60
    # 음수에서 양수로 바뀌는 모든 인덱스 찾기
    idx_20_60_dead_cross = [idx for idx in range(len(df_raw)) if df_raw["MA20-60"].iloc[idx] < 0 and df_raw["MA20-60"].iloc[idx - 1] >= 0]


    df_raw_anal.loc[:, '5_20_cross'] = '-'
    df_raw_anal.loc[idx_5_20_gold_cross,'5_20_cross'] = '골든크로스(매수)'
    df_raw_anal.loc[idx_5_20_dead_cross,'5_20_cross'] = '데드크로스(매도)'

    df_raw_anal.loc[:, '20_60_cross'] = '-'
    df_raw_anal.loc[idx_20_60_gold_cross,'20_60_cross'] = '골든크로스(매수)'
    df_raw_anal.loc[idx_20_60_dead_cross,'20_60_cross'] = '데드크로스(매도)'


    # 정배열 역배열
    ascending_sq  = (df_raw['MA5-20'] > 0) & \
    (df_raw['MA20-60'] > 0) & \
    (df_raw['MA60-120'] > 0)

    descending_sq  = (df_raw['MA5-20'] < 0) & \
    (df_raw['MA20-60'] < 0) & \
    (df_raw['MA60-120'] < 0)

    df_raw_anal.loc[:,'array'] = '-'
    df_raw_anal.loc[ascending_sq,'array'] = '정배열(매수)'
    df_raw_anal.loc[descending_sq,'array'] = '역배열(매도)'


    # 볼린저밴드
    df_raw['close'] = df_raw['close'].astype('float64')
    down_reg_sq = df_raw['upper'] - df_raw['close']
    top_reg_sq  = df_raw['lower'] - df_raw['close']

    down_reg = [idx for idx in range(1,len(df_raw)) if down_reg_sq[idx] > 0 and down_reg_sq[idx-1] <= 0]
    top_reg = [idx for idx in range(1,len(df_raw)) if top_reg_sq[idx] < 0 and top_reg_sq[idx-1] >= 0]

    df_raw_anal.loc[:,'Bollinger_band'] = '-'
    df_raw_anal.loc[down_reg,'Bollinger_band'] = '하향회귀(매도)'
    df_raw_anal.loc[top_reg,'Bollinger_band'] = '상향회귀(매수)'


    # MACD
    signal_down_cross = [idx for idx in range(1,len(df_raw)) if df_raw['MACD_DIFF'][idx] < 0 and df_raw['MACD_DIFF'][idx-1] >= 0]
    signal_top_corss = [idx for idx in range(1,len(df_raw)) if df_raw['MACD_DIFF'][idx] > 0 and df_raw['MACD_DIFF'][idx-1] <= 0]

    df_raw_anal.loc[:,'MACD'] = '-'
    df_raw_anal.loc[signal_down_cross,'MACD'] = '하향돌파(매도)'
    df_raw_anal.loc[signal_top_corss,'MACD'] = '상향돌파(매수)'

    # RSI
    down_reg = [idx for idx in range(1,len(df_raw)) if df_raw['RSI'][idx] > 70 and df_raw['RSI'][idx-1] <= 70]
    top_reg = [idx for idx in range(1,len(df_raw)) if df_raw['RSI'][idx] < 30 and df_raw['RSI'][idx-1] >= 30]


    df_raw_anal.loc[:,'RSI'] = '-'
    df_raw_anal.loc[down_reg,'RSI'] = 'RSI 상단 하향돌파(매도)'
    df_raw_anal.loc[top_reg,'RSI'] = 'RSI 하단 상향 돌파(매수)'


    df_raw_total = pd.concat([df_raw_total, df_raw])
    df_raw_anal_total = pd.concat([df_raw_anal_total, df_raw_anal])

    print(ticker_nm)


df_raw_total = df_raw_total.reset_index(drop = True)
df_raw_anal_total = df_raw_anal_total.reset_index(drop = True)

001045
001530
001040
079160
005830


In [251]:
df_raw_total.head(3)

,date,open,high,low,close,volume,price_change_percentage,ticker,corp_name,market,...,MACD_Signal,변화량,상승폭,하락폭,AU,AD,RSI,MA5-20,MA20-60,MA60-120
0,2018-01-02 00:00:00+00:00,68519,68519,66664,66930.0,2522,-1.686301,001045,CJ우,KOSPI,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-03 00:00:00+00:00,68254,68254,66488,67106.0,2501,0.262961,001045,CJ우,KOSPI,...,NaN,176.0,176.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-04 00:00:00+00:00,67636,68077,65870,66665.0,8353,-0.657169,001045,CJ우,KOSPI,...,NaN,-441.0,0.0,441.0,NaN,NaN,NaN,NaN,NaN,NaN


In [250]:
df_raw_anal_total.head(3)

,date,ticker,corp_name,market,close,5_20_cross,20_60_cross,array,Bollinger_band,MACD,RSI
0,2018-01-02 00:00:00+00:00,001045,CJ우,KOSPI,66930,-,-,-,-,-,-
1,2018-01-03 00:00:00+00:00,001045,CJ우,KOSPI,67106,-,-,-,-,-,-
2,2018-01-04 00:00:00+00:00,001045,CJ우,KOSPI,66665,-,-,-,-,-,-


In [ ]:

df_raw_total_2 = df_raw_total[df_raw_total['date'] > '2023-01-01'].reset_index(drop = True)
df_raw_anal_total_2 = df_raw_anal_total[df_raw_anal_total['date'] > '2023-01-01'].reset_index(drop = True)

In [243]:

for market_nm in ['KOSPI', 'KOSDAQ']:
    df_raw_total_3 = df_raw_total_2[df_raw_total_2['market'] == market_nm].reset_index(drop = True)
    df_raw_anal_total_3 = df_raw_anal_total_2[df_raw_anal_total_2['market'] == market_nm].reset_index(drop = True)

    table_from_pandas = pa.Table.from_pandas(df_raw_total_3,preserve_index = False)
    pq.write_table(table_from_pandas, f'data_crawler/cleaning/kor_stock_ohlcv/df_raw_total_2_{market_nm}_{today_date1}.parquet')

    table_from_pandas = pa.Table.from_pandas(df_raw_anal_total_3,preserve_index = False)
    pq.write_table(table_from_pandas, f'data_crawler/cleaning/kor_stock_ohlcv/df_raw_anal_total_2_{market_nm}_{today_date1}.parquet')
    

    # Google Storage 적재
    source_file_name = f'data_crawler/cleaning/kor_stock_ohlcv/df_raw_total_2_{market_nm}_{today_date1}.parquet'    # GCP에 업로드할 파일 절대경로
    destination_blob_name = f'data_crawler/cleaning/kor_stock_ohlcv/df_raw_total_2_{market_nm}_{today_date1}.parquet'    # 업로드할 파일을 GCP에 저장할 때의 이름
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)


    # Google Storage 적재
    source_file_name = f'data_crawler/cleaning/kor_stock_ohlcv/df_raw_anal_total_2_{market_nm}_{today_date1}.parquet'    # GCP에 업로드할 파일 절대경로
    destination_blob_name = f'data_crawler/cleaning/kor_stock_ohlcv/df_raw_anal_total_2_{market_nm}_{today_date1}.parquet'    # 업로드할 파일을 GCP에 저장할 때의 이름
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)

,date,ticker,corp_name,market,close,5_20_cross,20_60_cross,array,Bollinger_band,MACD,RSI
0,2023-01-02 00:00:00+00:00,001045,CJ우,KOSPI,49300,데드크로스(매도),-,-,-,-,-
1,2023-01-03 00:00:00+00:00,001045,CJ우,KOSPI,49000,-,-,-,-,-,-
2,2023-01-04 00:00:00+00:00,001045,CJ우,KOSPI,48650,-,-,-,-,-,-
3,2023-01-05 00:00:00+00:00,001045,CJ우,KOSPI,48200,-,-,-,-,-,-
4,2023-01-06 00:00:00+00:00,001045,CJ우,KOSPI,48350,-,-,-,상향회귀(매수),-,-
...,...,...,...,...,...,...,...,...,...,...,...
1240,2023-12-28 00:00:00+00:00,005830,DB손해보험,KOSPI,83700,-,-,-,-,-,-
1241,2024-01-02 00:00:00+00:00,005830,DB손해보험,KOSPI,81600,-,-,-,-,-,-
1242,2024-01-03 00:00:00+00:00,005830,DB손해보험,KOSPI,80900,-,-,-,-,-,-
1243,2024-01-04 00:00:00+00:00,005830,DB손해보험,KOSPI,79200,-,-,-,-,-,-


In [254]:
def rank_per(df1, rank_count):
    max_value_1 = df1[df1['market'] == 'KOSPI'].sort_values(by ='price_change_percentage', ascending = False).head(rank_count)
    max_value_2 = df1[df1['market'] == 'KOSDAQ'].sort_values(by ='price_change_percentage', ascending = False).head(rank_count)
    min_value_1 = df1[df1['market'] == 'KOSPI'].sort_values(by ='price_change_percentage', ascending = True).head(rank_count)
    min_value_2 = df1[df1['market'] == 'KOSDAQ'].sort_values(by ='price_change_percentage', ascending = True).head(rank_count)
    
    total = pd.concat([max_value_1, max_value_2, 
               min_value_1, min_value_2
              ])
    total = total.reset_index(drop = True)
    return total

### 주가 등락 하루

In [261]:
df = stock.get_market_ohlcv("20240105", market="ALL")

In [262]:
df

,시가,고가,저가,종가,거래량,거래대금,등락률
티커,,,,,,,
060310,3210,3325,3190,3300,1768963,5774925915,0.92
095570,5270,5290,5070,5280,410628,2137622110,1.15
006840,16950,17090,16950,16960,3005,51105470,-0.06
054620,6900,7460,6900,7300,112058,817598270,5.19
265520,23250,23400,22700,22950,104119,2386649500,-1.08
...,...,...,...,...,...,...,...
000540,3000,3030,2970,3000,56853,170400425,-0.50
000545,5640,5800,5640,5710,7264,41029090,1.24
003280,3185,3220,2835,2850,23883232,71191936640,-12.44


In [263]:
df2 = rank_per(ohlcv_df_raw[ohlcv_df_raw['date'] == '2024-01-05'], 5)


In [264]:
df2

,date,open,high,low,close,volume,price_change_percentage,ticker,corp_name,market
0,2024-01-05 00:00:00+00:00,14780,15480,14780,15480,62790,29.974811,000227,유유제약2우B,KOSPI
1,2024-01-05 00:00:00+00:00,6800,7590,6500,6800,2036217,16.438356,000225,유유제약1우,KOSPI
2,2024-01-05 00:00:00+00:00,2250,2685,2190,2525,2137698,14.512472,001140,국보,KOSPI
3,2024-01-05 00:00:00+00:00,22050,24650,22000,24650,346454,11.286682,111380,동인기연,KOSPI
4,2024-01-05 00:00:00+00:00,7600,8220,6990,6990,2440916,10.426540,009415,태영건설우,KOSPI
5,2024-01-05 00:00:00+00:00,331,331,303,303,4050828,-19.200000,168490,한국패러랠,KOSPI
6,2024-01-05 00:00:00+00:00,8300,8300,7700,7700,181920,-16.122004,36328K,티와이홀딩스우,KOSPI
7,2024-01-05 00:00:00+00:00,12480,12590,12000,12020,2189476,-14.812190,004090,한국석유,KOSPI
8,2024-01-05 00:00:00+00:00,3185,3220,2835,2850,23578694,-12.442396,003280,흥아해운,KOSPI
9,2024-01-05 00:00:00+00:00,2040,2730,1946,2025,7976602,-12.337662,019685,대교우B,KOSPI


In [29]:
kor_stock_ohlcv_005930 = kor_stock_ohlcv_2[kor_stock_ohlcv_2['ticker'] == '005930'].reset_index()

In [30]:
kor_stock_ohlcv_005930

,index,date,open,high,low,close,volume,trading_value,price_change_percentage,ticker,corp_name,market
0,572165,2018-01-02,51380,51400,50780,51020,169485,432677351468,0.12,005930,삼성전자,KOSPI
1,572166,2018-01-03,52540,52560,51420,51620,200270,518345810160,1.18,005930,삼성전자,KOSPI
2,572167,2018-01-04,52120,52180,50640,51080,233909,600531577700,-1.05,005930,삼성전자,KOSPI
3,572168,2018-01-05,51300,52120,51200,52120,189623,490792925116,2.04,005930,삼성전자,KOSPI
4,572169,2018-01-08,52400,52520,51500,52020,167673,435974098536,-0.19,005930,삼성전자,KOSPI
...,...,...,...,...,...,...,...,...,...,...,...,...
1365,573530,2023-07-17,73200,73500,72800,73300,10060049,736819227400,-0.14,005930,삼성전자,KOSPI
1366,573531,2023-07-18,73200,73500,72000,72000,11697900,847564074902,-1.77,005930,삼성전자,KOSPI
1367,573532,2023-07-19,72700,72800,71300,71700,10896412,782147498048,-0.42,005930,삼성전자,KOSPI
1368,573533,2023-07-20,71100,71500,70800,71000,9732730,692487601800,-0.98,005930,삼성전자,KOSPI


### 주가 등락률 일주일

In [ ]:
diff_date = 60
now = datetime.now()
now = now + timedelta(days=-diff_date)
set_date = now.strftime('%Y-%m-%d')
df1 =  ohlcv_df_raw[ohlcv_df_raw['date'] > set_date].groupby("ticker")['date'].min().reset_index()



diff_date_list = [30, 90, 180, 240, 365]
now = datetime.now()
set_date = now.strftime('%Y-%m-%d')

ohlcv_df_raw['date'] = pd.to_datetime(ohlcv_df_raw['date'])


df1_set =  ohlcv_df_raw[ohlcv_df_raw['date'] > '2023-10-01'].groupby("ticker")['date'].max().reset_index()
# df1_set =  ohlcv_df_raw[ohlcv_df_raw['date'] == '2023-10-06'].groupby("ticker")['date'].max().reset_index()
df1_set = df1_set[['ticker', 'date']]
df1_set = pd.merge(df1_set, ohlcv_df_raw[['ticker', 'date', 'close', 'corp_name']],
              on = ['ticker', 'date'], 
              how = 'left')

    
for diff_date in diff_date_list:
# diff_date = 240
    now = datetime.now()
    now = now + timedelta(days=-diff_date)
    set_date = now.strftime('%Y-%m-%d')
    df1 =  ohlcv_df_raw[ohlcv_df_raw['date'] > set_date].groupby("ticker")['date'].min().reset_index()

    
    df1 = df1[['ticker','date']]
    df1 = pd.merge(df1, ohlcv_df_raw[['ticker', 'date', 'close']],
                  on = ['ticker', 'date'], 
                  how = 'left')
    
    df1.columns = ['ticker', f'date_{diff_date}', f'close_{diff_date}']

    df1_set = pd.merge(df1_set, df1, 
                      on = 'ticker', 
                      how = 'left')

    df1_set[f'per_{diff_date}'] = (df1_set['close'] - df1_set[f'close_{diff_date}']) / df1_set[f'close_{diff_date}']    


df1_set_2 = df1_set[['date','ticker','per_30', 'per_90', 'per_180', 'per_240', 'per_365', 'corp_name']]

df_per_total = pd.DataFrame()
per_set = ['per_30', 'per_90', 'per_180', 'per_240', 'per_365']
ascending_list = [True, False]

for per_value in per_set:
    for ascending_value in ascending_list:
        df_per =  df1_set_2.sort_values(by =per_value, ascending = ascending_value).head()
        df_per['type'] = per_value
        df_per['rank'] = range(1,6)
        df_per_total = pd.concat([df_per_total, df_per])

df_per_total = df_per_total.drop_duplicates()        

### 주가 등락 한달

### 주가 등락 1년